In [3]:
import os

In [4]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML\\research'

In [5]:
os.chdir("../")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from wine_pred.constants import *

from wine_pred.utils.common import read_yaml, create_directories
 

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


        

In [9]:
import urllib.request as request
import zipfile
from wine_pred.logging.logger import logger
from wine_pred.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2024-07-03 10:53:45,242 ] 29 root - INFO - yaml file: config\config.yaml loaded succesfully
[ 2024-07-03 10:53:45,243 ] 29 root - INFO - yaml file: params.yaml loaded succesfully
[ 2024-07-03 10:53:45,246 ] 29 root - INFO - yaml file: schema.yaml loaded succesfully
[ 2024-07-03 10:53:45,246 ] 43 root - INFO - Created directory at: artifacts
[ 2024-07-03 10:53:45,247 ] 43 root - INFO - Created directory at: artifacts/data_ingestion
[ 2024-07-03 10:53:46,507 ] 11 root - INFO - artifacts/data_ingestion/wine_quality.zip downloaded! with following info: 
Connection: close
Content-Length: 21994
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "749b9f1a5fdf247146f74dc0036227d83a2bf05bebb0bbfe113c02012d0703d2"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 872B:3700EB:2FB89:4A8D9:6684DCDE
Accept-R